In [1]:
import pandas as pd
from IPython.display import display
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import mannwhitneyu
import gc
import numpy as np

In [2]:
# df = pd.read_csv("c:\\data\\e-commerce\\2019-Oct.csv", usecols=['event_time', 'event_type', 'product_id', 'brand', 'category_code', 'price', 'user_id', 'user_session'])
# df.head(2)

In [3]:
# df['category_code'] = df['category_code'].astype('category')
# df['category_code'] = df['category_code'].cat.add_categories('unknown.')
# df['category_code'].fillna('unknown.', inplace=True)

# df['general_category'] = df['category_code'].apply(lambda x: x.split('.')[0])
# display(df.head(2))

In [4]:
# df['user_session'] = df['user_session'].astype('category').cat.codes
# df.head()

In [5]:
# df.head(2)

In [6]:
# # datetime 처리
# df['event_time'] = pd.to_datetime(df['event_time'])
# df['date'] = df['event_time'].dt.date

# # 사용자별, 시간순 정렬
# df = df.sort_values(['user_id', 'event_time'])

# # 구매 발생 플래그
# df['is_purchase'] = df['event_type'] == 'purchase'

# # 하루 단위 기준 세션 초기화
# df['daily_session'] = df['user_id'].astype(str) + '_' + df['date'].astype(str)

# # 구매 발생 시 세션 쪼개기: 구매 이후는 새로운 세션으로
# def split_on_purchase(group):
#     session_id = []
#     counter = 0
#     for is_p in group['is_purchase']:
#         session_id.append(counter)
#         if is_p:
#             counter += 1
#     return session_id

# # 사용자 + 날짜별로 쪼개서 세션 나누기
# df['session_split_id'] = df.groupby(['user_id', 'date']).apply(split_on_purchase).explode().values

# # reallyreal진짜최종의최총의최최종
# df['custom_session'] = (
#     df['user_id'].astype(str) + '_' +
#     df['date'].astype(str) + '_' +
#     df['session_split_id'].astype(str)
# )
# df.head(2)


In [7]:
# df.to_csv("c:\\data\\e-commerce\\custom_session.csv", index=False) # event_time, event_type, product_id, category_code, brand, price, user_id, user_session, general_category, buy, date, is_purchase, daily_session, session_split_id, custom_session
custom_session = pd.read_csv("c:\\data\\e-commerce\\custom_session.csv", usecols=['event_time', 'event_type', 'product_id', 'category_code', 'brand', 'price', 'user_id', 'general_category', 'buy', 'custom_session']);custom_session.head(2)
custom_session.head(2)

,event_time,event_type,product_id,category_code,brand,price,user_id,general_category,buy,custom_session
0,2019-10-23 20:04:08+00:00,view,7002639,kids.carriage,bumbleride,769.65,33869381,kids,0,33869381_2019-10-23_0
1,2019-10-13 00:13:46+00:00,view,10600284,unknown.,NaN,0.00,64078358,unknown,0,64078358_2019-10-13_0


In [8]:
buy_sessions = custom_session[custom_session['event_type'] == 'purchase']['custom_session'].unique()

buy_log = custom_session[custom_session['custom_session'].isin(buy_sessions)]
not_buy_log = custom_session[~custom_session['custom_session'].isin(buy_sessions)]
custom_session['buy'] = custom_session['custom_session'].isin(buy_log['custom_session']).astype(int)
custom_session.head(2)

,event_time,event_type,product_id,category_code,brand,price,user_id,general_category,buy,custom_session
0,2019-10-23 20:04:08+00:00,view,7002639,kids.carriage,bumbleride,769.65,33869381,kids,0,33869381_2019-10-23_0
1,2019-10-13 00:13:46+00:00,view,10600284,unknown.,NaN,0.00,64078358,unknown,0,64078358_2019-10-13_0


In [9]:
median_price_category = custom_session.groupby('general_category')['price'].median().to_dict()
custom_session['relative_price'] = custom_session['price'] / custom_session['general_category'].map(median_price_category)
custom_session.head(2)

,event_time,event_type,product_id,category_code,brand,price,user_id,general_category,buy,custom_session,relative_price
0,2019-10-23 20:04:08+00:00,view,7002639,kids.carriage,bumbleride,769.65,33869381,kids,0,33869381_2019-10-23_0,6.986022
1,2019-10-13 00:13:46+00:00,view,10600284,unknown.,NaN,0.00,64078358,unknown,0,64078358_2019-10-13_0,0.000000


In [10]:
custom_session['custom_session'] = custom_session['custom_session'].astype('category').cat.codes
custom_session.head(2)

,event_time,event_type,product_id,category_code,brand,price,user_id,general_category,buy,custom_session,relative_price
0,2019-10-23 20:04:08+00:00,view,7002639,kids.carriage,bumbleride,769.65,33869381,kids,0,431,6.986022
1,2019-10-13 00:13:46+00:00,view,10600284,unknown.,NaN,0.00,64078358,unknown,0,42508,0.000000


In [11]:
def log_dictionary(df):

    df = df.sort_values(['user_id', 'event_time'])
    df['event_time'] = pd.to_datetime(df['event_time'])

    session_data = []

    for session, group in df.groupby('custom_session'):
        group = group.sort_values('event_time')

        times = group['event_time'].tolist()
        time_since_start = [(t - times[0]).total_seconds() for t in times]
        time_since_prev = [0] + [(times[i] - times[i-1]).total_seconds() for i in range(1, len(times))]

        events = []
        for _, row in group.iterrows():
            if row['event_type'] == 'purchase':
                continue  # 구매 제외
            
            events.append({
                "event_type": row["event_type"],
                "product_id": row["product_id"],
                "category_code": row["category_code"] if pd.notna(row["category_code"]) else "unknown.",
                "general_category": row["general_category"] if pd.notna(row["general_category"]) else "unknown",
                "brand": row["brand"] if pd.notna(row["brand"]) else "unknown",
                "price": row["price"],
                "relative_price": row["relative_price"],
                "time_since_start": time_since_start[len(events)],
                "time_since_prev": time_since_prev[len(events)]
            })

        session_data.append({
            "session": session,
            "event": events,
            "buy": int(group["buy"].max())
        })

    return pd.DataFrame(session_data)

In [12]:
log_dict = log_dictionary(custom_session)

In [13]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None) 

# pd.reset_option("display.max_colwidth")
# pd.reset_option("display.max_rows")
# pd.reset_option("display.max_columns")


In [14]:
display(log_dict.head(2))

,session,event,buy
0,0,"[{'event_type': 'view', 'product_id': 22200103, 'category_code': 'unknown.', 'general_category': 'unknown', 'brand': 'unknown', 'price': 15.77, 'relative_price': 0.17504717504717504, 'time_since_start': 0.0, 'time_since_prev': 0}]",0
1,1,"[{'event_type': 'view', 'product_id': 6902133, 'category_code': 'furniture.living_room.chair', 'general_category': 'furniture', 'brand': 'joie', 'price': 143.89, 'relative_price': 0.800857126954973, 'time_since_start': 0.0, 'time_since_prev': 0}, {'event_type': 'view', 'product_id': 6902133, 'category_code': 'furniture.living_room.chair', 'general_category': 'furniture', 'brand': 'joie', 'price': 143.89, 'relative_price': 0.800857126954973, 'time_since_start': 41.0, 'time_since_prev': 41.0}, {'event_type': 'view', 'product_id': 6902303, 'category_code': 'furniture.living_room.chair', 'general_category': 'furniture', 'brand': 'joie', 'price': 111.46, 'relative_price': 0.6203595480603329, 'time_since_start': 373.0, 'time_since_prev': 332.0}, {'event_type': 'view', 'product_id': 6902303, 'category_code': 'furniture.living_room.chair', 'general_category': 'furniture', 'brand': 'joie', 'price': 111.46, 'relative_price': 0.6203595480603329, 'time_since_start': 387.0, 'time_since_prev': 14.0}]",0


In [15]:
from sklearn.preprocessing import LabelEncoder
le_event = LabelEncoder()
le_event.fit(custom_session['event_type'].unique())
le_event.classes_

array(['cart', 'purchase', 'view'], dtype=object)

In [16]:
def encode_event_type(events):
    for event in events:
        event['event_type'] = le_event.transform([event['event_type']])[0]
    return events

# 적용
log_dict['event'] = log_dict['event'].apply(encode_event_type)

In [17]:
display(log_dict.head(2))

,session,event,buy
0,0,"[{'event_type': 2, 'product_id': 22200103, 'category_code': 'unknown.', 'general_category': 'unknown', 'brand': 'unknown', 'price': 15.77, 'relative_price': 0.17504717504717504, 'time_since_start': 0.0, 'time_since_prev': 0}]",0
1,1,"[{'event_type': 2, 'product_id': 6902133, 'category_code': 'furniture.living_room.chair', 'general_category': 'furniture', 'brand': 'joie', 'price': 143.89, 'relative_price': 0.800857126954973, 'time_since_start': 0.0, 'time_since_prev': 0}, {'event_type': 2, 'product_id': 6902133, 'category_code': 'furniture.living_room.chair', 'general_category': 'furniture', 'brand': 'joie', 'price': 143.89, 'relative_price': 0.800857126954973, 'time_since_start': 41.0, 'time_since_prev': 41.0}, {'event_type': 2, 'product_id': 6902303, 'category_code': 'furniture.living_room.chair', 'general_category': 'furniture', 'brand': 'joie', 'price': 111.46, 'relative_price': 0.6203595480603329, 'time_since_start': 373.0, 'time_since_prev': 332.0}, {'event_type': 2, 'product_id': 6902303, 'category_code': 'furniture.living_room.chair', 'general_category': 'furniture', 'brand': 'joie', 'price': 111.46, 'relative_price': 0.6203595480603329, 'time_since_start': 387.0, 'time_since_prev': 14.0}]",0
